In [2]:
using Oscar

  ___   ____   ____    _    ____
 / _ \ / ___| / ___|  / \  |  _ \   |  Combining ANTIC, GAP, Polymake, Singular
| | | |\___ \| |     / _ \ | |_) |  |  Type "?Oscar" for more information
| |_| | ___) | |___ / ___ \|  _ <   |  Manual: https://docs.oscar-system.org
 \___/ |____/ \____/_/   \_\_| \_\  |  Version 1.0.3


# Galois cohomology and the Shafarevich–Weil theorem

The study of the action of Galois groups on arithmetic objects has a long history in number theory. One classical application is the construction and investigation of number fields with prescribed invariants using Galois cohomology. In this regard, we will show how to use Oscar to determine fundamental classes of normal extensions and determine Galois groups using the Shafarevich–Weil theorem.

Our running example will be the number field

$$ k = \mathbf{Q}(\alpha) = \mathbf{Q}[X]/(x^4 - 12x^3 + 36x^2 - 36x + 9) $$

We begin by defining the field:

In [6]:
QQx, x = QQ["x"];

In [7]:
k, α = number_field(x^4 - 12*x^3 + 36*x^2 - 36*x + 9, "α");

### Galois group

Fundamental to any computation is with Galois cohomology is the computation of the Galois group itself.

In [9]:
Gk, Gkctx = galois_group(k)

(Permutation group of degree 4 and order 4, Galois context for x^4 - 12*x^3 + 36*x^2 - 36*x + 9 and prime 11)

Note that this not only returns Galois group as a permutation group on $4$ letters, but also approximations of the roots (in this case, in a splitting field $\mathbf{Q}_{11}$. The group `Gk` is an ordinary Oscar group and thus can be investigated using `describe`:

In [11]:
describe(Gk)

"C2 x C2"

Thus the $k/\mathbf{Q}$ is normal with Galois group the Klein four-group $\operatorname{Gal}(k/\mathbf{Q}) \cong \mathrm{C}_2 \times \mathrm{C}_2$.

### Subfields

So there should be 3 quadratic subfields:

In [11]:
subs = subfields(k, degree = 2)

3-element Vector{Tuple{AbsSimpleNumField, NumFieldHom{AbsSimpleNumField, AbsSimpleNumField, Hecke.MapDataFromAnticNumberField{AbsSimpleNumFieldElem}, Hecke.MapDataFromAnticNumberField{AbsSimpleNumFieldElem}, AbsSimpleNumFieldElem}}}:
 (Number field of degree 2 over QQ, Map: number field -> k)
 (Number field of degree 2 over QQ, Map: number field -> k)
 (Number field of degree 2 over QQ, Map: number field -> k)

To identify them, we'll compute their discriminants...

In [13]:
[discriminant(maximal_order(x[1])) for x = subs]

3-element Vector{ZZRingElem}:
 12
 8
 24

### Constructing abelian extensions via class field theory

Next we want to study quadratic extensions of $k$. Our aim is to determine the maximal elementary abelian $2$-extension $K$ ramified only at $2$ and $3$, which is normal over $\mathbf{Q}$. To this end, we will use constructive class field theory and begin by determining the maximal abelian extension of $k$ ramified only at $2$ and $3$, that is, the ray class field $M$ with modulus $3 \cdot 8 \cdot \mathcal{O}_k$:

$$ \mathbf{Q} \subseteq k \subseteq K \subseteq M. $$

In [17]:
M = ray_class_field(3*8*ring_of_integers(k))

Class field defined mod (<24, 24>, InfPlc{AbsSimpleNumField, AbsSimpleNumFieldEmbedding}[]) of structure (Z/4)^2

In [18]:
a = filter(is_normal, subfields(a, degree = 2));

In [19]:
K = a[1]

Class field defined mod (<24, 24>, InfPlc{AbsSimpleNumField, AbsSimpleNumFieldEmbedding}[]) of structure Z/2

### Shafarevich–Weil theorem

Thus $K$ is a quadratic extension of $k$. Since $K/\mathbf{Q}$ is normal, we might ask how the Galois group $G = \operatorname{Gal}(K/\mathbf{Q})$ looks like. We know that it is an extension of $\operatorname{Gal}(k/\mathbf{Q})$ by $\operatorname{Gal}(K/k)$:

$$ 1 \to \operatorname{Gal}(K/k) \to \operatorname{Gal}(K/\mathbf{Q}) \to \operatorname{Gal}(k/\mathbf{Q}) $$

and thus determines (and is determined by) a class in $c \in \mathrm{H}^2(\operatorname{Gal}(k/\mathbf{Q}), \operatorname{Gak}(K/k))$. The celebrated *Shafarevich–Weil theorem* asserts that $c$ is equal to the image of the *fundmanteal class*

$$ \delta \in \mathrm{H}^2(\mathrm{Gal}(k/\mathbf{Q}), C_k) $$

under the canonical projection to the ray class group. Here, $C_k = \mathbf{A}_k^\times/k^\times$ is the *idèle class group* of $k$.

### Old

A class field is paramterized by a quotient of some ray class group. At this point, the no defining equation of the field has been computed. To compute the galois group (over Q), we use class field theory:
 - the group is an extension: `1 -> C_2 -> X -> C2 x C2 -> 1`
 - thus an element of `H^2(C2 x C2, C2)`
 - galois cohomology gives `H^2(C2 x C2, C) = C_4` where `C` denotes the idel class group.
 - there is a canonical generator `delta` for this
 - the idel class group projects onto any ray class group
 - `X` as an element of the `H^2` is obtained by projecting `delta` down

In [17]:
b = [galois_group(x, QQ) for x = a];

In [18]:
b[1]

(Permutation group of degree 8 and order 8, (2-cochain with values in Z/2, G-module for permutation group of degree 4 and order 4 acting on Z/2))

In [19]:
[describe(x[1]) for x = b]

3-element Vector{String}:
 "C4 x C2"
 "Q8"
 "D8"

### Cohomology of the idele group

Let's look at this in more detail. First we need to realize a $\operatorname{Gal}(k/\mathbf{Q})$-module representing the idéle class group:

In [23]:
I = idel_class_gmodule(k)

Idel-group for Number field of degree 4 over QQ using ZZRingElem[2, 3] as places

Now we can compute the cohomology (the idèle class group is apriori infinite, but has cohomologically trivial subgroups of finite index):

In [29]:
H2, mp, _ = cohomology_group(I, 2);

In [31]:
number_of_generators(H2)

151

By theory, this is cyclic of order $4$. To see this structure, we compute a canonical decomposition of this group:

In [33]:
S, ms = snf(H2); S

Z/4

We now get a generator of $\mathrm{H}^2(\operatorname{Gal}(k/\mathbf{Q}, C_k)$:

In [24]:
a = mp(ms(s[1]))

2-cochain with values in (Z/2)^4 x Z/8 x Z/16 x Z/32 x Z/128 x Z/512 x Z^(17)

In [25]:
G, mG = automorphism_group(PermGroup, k)

(Permutation group of degree 4 and order 4, Map: permutation group of degree 4 and order 4 -> generic group of order 4 with multiplication table -> set of automorphisms of Number field of degree 4 over QQ)

In [26]:
i = a(G[1], G[1])

Abelian group element [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 12, 242, 0, 0, 0, 12, -318, 0, 0, 0, 0, 560, 0, 0, 0, 0]

... but we can compute an ideal that would represent the same class - in a ray class group modulo 12:

In [27]:
ideal(I, ans, coprime = 12*maximal_order(k))

1//851777007454951063807395976805683743870799462999244449 * <851777007454951063807395976805683743870799462999244449, 688346858552416230003426443541737868176025810334911088337225957792407202703178853297336661423674459246723365*_a^3 + 214814237981429478342287826384260543321913599131038457295989337480394754821555899555784040126673066236013274*_a^2 + 255700829714150796156041822211320766853498877110948286769675365063343009872707834166704541691873530898502783*_a + 233775964634726252852867195375498105184410447718228493592424225582367266010479028913219735805505530803244190>
Norm: 617984721546473612969832186137884293349816144449690365970190470664087524997924867626178773664186884271497996352265750400048961683875900007333071688876974851450849
basis_matrix 
[851777007454951063807395976805683743870799462999244449 0 0 0; 0 851777007454951063807395976805683743870799462999244449 0 0; 0 0 851777007454951063807395976805683743870799462999244449 0; 450333948888262695609085896070228118825224696261521567 47

In [28]:
Oscar.GaloisCohomology_Mod.idel_finite(I, i)

Dict{AbsSimpleNumFieldOrderIdeal, Tuple{AbsSimpleNumFieldElem, Hecke.LocalFieldElem}} with 2 entries:
  <2, _a + 1>                   => (751254989337*_a^3 + 2750695505974*_a^2 + 42…
  <3, 7*_a^3 + _a^2 + 8*_a + 3> => (1, 1)

In [29]:
collect(values(ans))[1]

(751254989337*_a^3 + 2750695505974*_a^2 + 4207533492259*_a + 3265080393263, (2^0 + 2^3 + 2^4 + 2^9 + 2^11 + 2^13 + 2^14 + 2^16 + 2^19 + 2^22 + 2^25 + 2^27 + 2^29 + 2^30 + 2^31 + 2^33 + 2^34 + 2^35 + 2^37 + 2^39 + O(2^40))*a^3 + (2^0 + 2^1 + 2^3 + 2^5 + 2^6 + 2^7 + 2^8 + 2^14 + 2^17 + 2^18 + 2^19 + 2^20 + 2^22 + 2^24 + 2^25 + 2^28 + 2^29 + 2^31 + 2^32 + 2^33 + 2^36 + 2^37 + 2^38 + O(2^40))*a^2 + (2^1 + 2^9 + 2^10 + 2^14 + 2^16 + 2^20 + 2^21 + 2^22 + 2^23 + 2^25 + 2^26 + 2^27 + 2^28 + 2^29 + 2^30 + 2^32 + 2^33 + 2^34 + 2^35 + 2^36 + 2^38 + 2^39 + O(2^40))*a + 2^0 + 2^3 + 2^5 + 2^10 + 2^11 + 2^18 + 2^19 + 2^20 + 2^22 + 2^24 + 2^27 + 2^28 + 2^33 + 2^34 + 2^36 + 2^37 + 2^38 + 2^39 + O(2^40))

The 1st part is an element in `k` that has the 2nd element as value in the completion. 